In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os
import h5py as hdf
import scipy as scp

plt.rcParams['figure.facecolor'] = 'white'

In [14]:
def gather_universal(path: str,filetype: str)-> list:
    """
    :param path:
    :param filetype:
    :return:
    """
    subdir=path

    data=[]
    for path, subdirs, files in os.walk(subdir):
        for name in files:
            if name.endswith(filetype) and name[-5]!='t'  :

                b=(str(os.path.join(path, name)))
                data.append(b)
    return data

def gather_subdirs(paths: str)-> list:
    subdir=paths

    data=[]
    for path, subdirs, files in os.walk(subdir):
        for dir in subdirs:
            data.append(dir)
    return data

def sort_on_last(list:list)->list:
    df=pd.DataFrame({'list': list})
    df['nums']=df['list'].apply(lambda x:x.split('_')[-1].split('.')[0])
    df['nums']=df.nums.astype(int)
    df.sort_values(by='nums',inplace=True)
    #display(df)
    return(np.array(df['list']))
path='data/hdf/'
display(gather_universal(path,'.hdf5'))

def get_hdf_main_groups(path) -> list:
    with hdf.File(path,'r') as r:
        #display([str(i) for i in r.keys()])
        return [str(i) for i in r.keys()]

def get_hdf_group_datasets(path,groupkey) -> list:
    with hdf.File(path,'r') as r:
        #display([str(i) for i in r[groupkey].keys()])
        return [str(i) for i in r[groupkey].keys()]

def get_df_columns(path)-> pd.DataFrame:
    groups=get_hdf_main_groups(path)
    cols=[]
    for key in groups:
       cols=cols+ list(zip(list(np.full(len(get_hdf_group_datasets(path,key)),key)),
                 get_hdf_group_datasets(path,key)))

    #display(cols)
    columns=pd.MultiIndex.from_tuples(cols, names=('group', 'dataset'))
    #display(pd.DataFrame(columns=columns))
    return columns
def zero_to_nan(array):
    array[array==0.]=np.nan
    return(array)
def steps_to_df(seriespath,examplefilepath)->pd.DataFrame:
    path=seriespath
    series=pd.DataFrame(index=sort_on_last(gather_universal(path,'.hdf5')),columns=get_df_columns(examplefilepath))

    for file in series.index:
        with hdf.File(str(file),'r') as r:
            for location in series.columns:
                series.loc[file,location]=r[location[0]][location[1]][:]
    return series
#series=steps_to_df('Y:\Istra Repositories\ch-gr_exports\MC_A_8yi','data/hdf/series_step_115.hdf5')
display(gather_subdirs('Y:\Istra Repositories\ch-gr_exports'))
bases=pd.DataFrame(index=gather_subdirs('Y:\Istra Repositories\ch-gr_exports'),columns=['path','data'])
bases['path']='Y:\\Istra Repositories\\ch-gr_exports\\'+bases.index
display(bases)

[]

['MC_A_6xi',
 'MC_A_6xiii',
 'MC_A_6yi',
 'MC_A_6yii',
 'MC_A_6yiii',
 'MC_A_8xi',
 'MC_A_8xii',
 'MC_A_8xiii',
 'MC_A_8yi',
 'MC_A_8yii',
 'MC_B_6xi',
 'MC_B_6xii',
 'MC_B_6xiii',
 'MC_B_6yi',
 'MC_B_6yii',
 'MC_B_6yiii',
 'MC_B_8xi',
 'MC_B_8xii',
 'MC_B_8xiii',
 'MC_B_8yi',
 'MC_B_8yii',
 'MC_B_8yiii',
 'MC_GW_6xi',
 'MC_GW_6xii',
 'MC_GW_6xiii',
 'MC_GW_6yi',
 'MC_GW_6yii',
 'MC_GW_6yiii',
 'MC_GW_8xi',
 'MC_GW_8xii',
 'MC_GW_8yi',
 'MC_GW_8yii',
 'MC_GW_8yiii',
 'QL_A_6xi',
 'QL_A_6xii',
 'QL_A_6xiii',
 'QL_A_6yi',
 'QL_A_6yii',
 'QL_A_6yiii',
 'QL_A_8xi',
 'QL_A_8xii',
 'QL_A_8yi',
 'QL_A_8yiii',
 'QL_B_6xii',
 'QL_B_6xiii',
 'QL_B_6yi',
 'QL_B_6yii',
 'QL_B_8xi',
 'QL_B_8xii',
 'QL_B_8xiii',
 'QL_B_8yi',
 'QL_B_8yiii',
 'QL_GW_6xi',
 'QL_GW_6xii',
 'QL_GW_6xiii',
 'QL_GW_6yi',
 'QL_GW_6yii',
 'QL_GW_6yiii',
 'QL_GW_8xi',
 'QL_GW_8xii',
 'QL_GW_8xiii',
 'QL_GW_8yi',
 'QL_GW_8yii']

,path,data
MC_A_6xi,Y:\Istra Repositories\ch-gr_exports\MC_A_6xi,NaN
MC_A_6xiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6xiii,NaN
MC_A_6yi,Y:\Istra Repositories\ch-gr_exports\MC_A_6yi,NaN
MC_A_6yii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yii,NaN
MC_A_6yiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yiii,NaN
...,...,...
QL_GW_8xi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xi,NaN
QL_GW_8xii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xii,NaN
QL_GW_8xiii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xiii,NaN
QL_GW_8yi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8yi,NaN


In [15]:
def isnotnan(num):
    return num == num
def linefit(x,y):
    coeffs=np.polyfit(x,y,1)
    poly=np.poly1d(coeffs)
    newy=poly(x)
    return newy

def strip_background(array):
    df=pd.DataFrame(array)
    #before=array
    #bf=plt.matshow(before)
    #plt.colorbar(bf)

    after=df.copy()
    after[after==0.]=np.nan
    #display(after)

    after.dropna(how='all',axis=0,inplace=True)
    after.dropna(how='all',axis=1,inplace=True)
    #display(after)
    """af=plt.matshow(after.dropna(how='all',axis=0))
    plt.colorbar(af)"""
    #display(after)
    return(after)

def get_line(df):
    afters=df.copy()
    for column in afters.columns:
        for row in afters.index:
            if isnotnan(afters.loc[row,column]) :
                afters.loc[row,column]=row
    coeffs=np.polyfit(afters.columns,afters.mean(axis=0,skipna=True),1)
    return coeffs

def get_scan_range(df,slope):
    xmin=df.columns.min()
    xmax=df.columns.max()
    ymin=df.index.min()
    ymax=df.index.max()
    pairs=[(xmin,ymin),(xmin,ymax),(xmax,ymin),(xmax,ymax)]
    #display(pairs)
    ts=[pair[1]-(slope*pair[0]) for pair in pairs]
    return (round(min(ts)),round(max(ts)))

def create_striper(stripped_background:pd.DataFrame):
    after=stripped_background
    scan_range=get_scan_range(after,get_line(after)[0])
    scan_range=range(*scan_range,1)
    res=pd.DataFrame(columns=after.columns,index=scan_range)
    #display(res)
    m=get_line(after)[0]
    for t in res.index:
        for x in res.columns:
            res.loc[t,x]=math.ceil((m*x)+t)
        #display(res.loc[t,:].dtypes)
    for column in res:
        res[column]=res[column].astype(int)
    return res
def apply_striper(striper,original_array):
    array=original_array
    res=striper

    #res.reset_index(inplace=True,drop=True)

    #display(min(after.index),max(after.index))
    striped=res.loc[range(min(after.index),max(after.index)+1),:].copy()
    #display(striped,after.index)

    #display(striped)
    array=pd.DataFrame(array)
    #display(array)
    for column in striped.columns:

        for index in striped.index:

            striped.loc[index,column]=array.loc[striped.loc[index,column],column]
    #display(striped)
    striped[striped == 0.]=np.nan
    return striped

In [16]:
import scipy as scipy
import scipy.signal as sgy
def sgolay2d ( z, window_size, order, derivative=None):
    """
    """
    # number of terms in the polynomial expression
    n_terms = ( order + 1 ) * ( order + 2)  / 2.0

    if  window_size % 2 == 0:
        window_size=window_size+1
        #raise ValueError('window_size must be odd')

    if window_size**2 < n_terms:
        raise ValueError('order is too high for the window size')

    half_size = window_size // 2

    # exponents of the polynomial.
    # p(x,y) = a0 + a1*x + a2*y + a3*x^2 + a4*y^2 + a5*x*y + ...
    # this line gives a list of two item tuple. Each tuple contains
    # the exponents of the k-th term. First element of tuple is for x
    # second element for y.
    # Ex. exps = [(0,0), (1,0), (0,1), (2,0), (1,1), (0,2), ...]
    exps = [ (k-n, n) for k in range(order+1) for n in range(k+1) ]

    # coordinates of points
    ind = np.arange(-half_size, half_size+1, dtype=np.float64)
    dx = np.repeat( ind, window_size )
    dy = np.tile( ind, [window_size, 1]).reshape(window_size**2, )

    # build matrix of system of equation
    A = np.empty( (window_size**2, len(exps)) )
    for i, exp in enumerate( exps ):
        A[:,i] = (dx**exp[0]) * (dy**exp[1])

    # pad input array with appropriate values at the four borders
    new_shape = z.shape[0] + 2*half_size, z.shape[1] + 2*half_size
    Z = np.zeros( (new_shape) )
    # top band
    band = z[0, :]
    Z[:half_size, half_size:-half_size] =  band -  np.abs( np.flipud( z[1:half_size+1, :] ) - band )
    # bottom band
    band = z[-1, :]
    Z[-half_size:, half_size:-half_size] = band  + np.abs( np.flipud( z[-half_size-1:-1, :] )  -band )
    # left band
    band = np.tile( z[:,0].reshape(-1,1), [1,half_size])
    Z[half_size:-half_size, :half_size] = band - np.abs( np.fliplr( z[:, 1:half_size+1] ) - band )
    # right band
    band = np.tile( z[:,-1].reshape(-1,1), [1,half_size] )
    Z[half_size:-half_size, -half_size:] =  band + np.abs( np.fliplr( z[:, -half_size-1:-1] ) - band )
    # central band
    Z[half_size:-half_size, half_size:-half_size] = z

    # top left corner
    band = z[0,0]
    Z[:half_size,:half_size] = band - np.abs( np.flipud(np.fliplr(z[1:half_size+1,1:half_size+1]) ) - band )
    # bottom right corner
    band = z[-1,-1]
    Z[-half_size:,-half_size:] = band + np.abs( np.flipud(np.fliplr(z[-half_size-1:-1,-half_size-1:-1]) ) - band )

    # top right corner
    band = Z[half_size,-half_size:]
    Z[:half_size,-half_size:] = band - np.abs( np.flipud(Z[half_size+1:2*half_size+1,-half_size:]) - band )
    # bottom left corner
    band = Z[-half_size:,half_size].reshape(-1,1)
    Z[-half_size:,:half_size] = band - np.abs( np.fliplr(Z[-half_size:, half_size+1:2*half_size+1]) - band )

    # solve system and convolve
    if derivative == None:
        m = np.linalg.pinv(A)[0].reshape((window_size, -1))
        """d1=pd.DataFrame(np.linalg.pinv(A)[1].reshape((window_size, -1)))
        d2=pd.DataFrame(np.linalg.pinv(A)[2].reshape((window_size, -1)))
        p=pd.DataFrame(m)
        display(p,p.shape,d1,d1.shape)
        fig,[a,b,c]=plt.subplots(1,3,figsize=(20,15))
        a.matshow(p, label='strain')
        b.matshow(d1,label='strain')
        c.matshow(d2,label='strain')

        X,Y=np.meshgrid(p.columns,p.index)
        fig = plt.figure(figsize=[20,15])
        ax = fig.gca(projection='3d')
        axis = ax.plot_surface(X,Y,p, cmap=cm.viridis,linewidth=5, antialiased=False)
        ax.view_init(30,-50)
        plt.colorbar(axis)"""

        return scipy.signal.fftconvolve(Z, m, mode='valid')
    elif derivative == 'col':
        c = np.linalg.pinv(A)[1].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -c, mode='valid')
    elif derivative == 'row':
        r = np.linalg.pinv(A)[2].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -r, mode='valid')
    elif derivative == 'both':
        c = np.linalg.pinv(A)[1].reshape((window_size, -1))
        r = np.linalg.pinv(A)[2].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -r, mode='valid'), scipy.signal.fftconvolve(Z, -c, mode='valid')

In [17]:
def perform_display(array):
    after=strip_background(array)
    res=create_striper(after)
    striped=apply_striper(res,array)
    """plt.matshow(striped)
    imf=plt.matshow((sgolay2d(striped.fillna(0).values,5,3)),cmap='plasma')

    plt.colorbar(imf)"""
    display(striped.shape)
    return striped.fillna(0)

def get_endsize(df):
    after=strip_background(df)
    #display(after)
    return (after.index[0],after.index[-1]),(after.columns[0],after.columns[-1])
def get_measure_length(df,col1,col2):
    mean_col1=df.loc[:,col1].mean()
    mean_col2=df.loc[:,col2].mean()
    return mean_col2-mean_col1

def get_macro_strain(current_df,reference_df):
    size=get_endsize(current_df)
    l_0=get_measure_length(reference_df,size[1][0],size[1][1])
    l=get_measure_length(current_df,size[1][0],size[1][1])
    dl=l-l_0
    return dl/l_0


def counter(df,sections):
    df=df.unstack().reset_index()
    df.columns=['X','Y','Z']
    resdict={}
    for i in range(len(sections)-1):
        resdict[sections[i]]=len(df[(df['Z']<=sections[i+1]) & (df['Z']>=sections[i])])
    #display(resdict)
    return(resdict)


In [18]:
#tryase=steps_to_df('Y:\Istra Repositories\ch-gr_exports\MC_GW_8xi','Y:\Istra Repositories\ch-gr_exports\MC_GW_8xi/series_step_115.hdf5')

In [19]:
"""#display(tryase)
bases.at['MC_GW_8xi','data']=tryase
display(bases.loc[:,'data'])"""

"#display(tryase)\nbases.at['MC_GW_8xi','data']=tryase\ndisplay(bases.loc[:,'data'])"

In [20]:
def create_spaced (num):
    return [round(x) for x in np.linspace(0,num,10)]



In [21]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

def mean_var_plot(ax,df):
    mean=[np.nanmean(val.values) for val in df]
    var=[np.nanvar(val.values) for val in df]
    ax.plot(var)
    #plt.errorbar(range(len(mean)),mean,yerr=var)
    plt.show()
def create_multiplot (bases,basename):
    typdf=bases.loc[basename,'data']
    print(type(typdf))
    locs=create_spaced(len(typdf.index)-1)
    cm=1/2.54
    fig,axs=plt.subplots(12,figsize=(16*cm,25*cm))
    fig.suptitle(basename, fontsize=18)
    variance=[]
    for i,loc in enumerate(locs):
        #display(type(strip_background(typdf.iloc[loc,:][('strains','strain_p1')])))
        axs[i].imshow(strip_background(typdf.iloc[loc,:][('strains','strain_p1')]),extent=(
            int(strip_background(typdf.iloc[loc,:][('strains','strain_p1')]).columns[0]),
            int(strip_background(typdf.iloc[loc,:][('strains','strain_p1')]).columns[-1]),
            int(strip_background(typdf.iloc[loc,:][('strains','strain_p1')]).index[0]),
            int(strip_background(typdf.iloc[loc,:][('strains','strain_p1')]).index[-1])


        ))
        variance.append(np.nanmean(strip_background(typdf.iloc[loc,:][('strains','strain_p1')])))
    axs[11].plot(range(len(typdf.index)),typdf[('add_data','analog_channels')])
    axs[10].plot(range(len(variance)),variance)
    for ax in axs:
        ax.xaxis.set_minor_locator(MultipleLocator(2.5))

    #fig.subplots_adjust(left=None, bottom=0, right=None, top=1.5, wspace=None, hspace=None)
    plt.show()
    expname=basename+'.png'
    fig.savefig(expname,dpi=500)



In [22]:
"""display(bases)
first_few_bases=bases.iloc[:10,:]
display(first_few_bases)



first_few_bases['data']=first_few_bases.path.apply(lambda x:steps_to_df(x,x+'\\series_step_0.hdf5'))
display(first_few_bases.columns)


#display(first_few_bases)
for index in first_few_bases.index:
    create_multiplot(first_few_bases,index)
"""

"display(bases)\nfirst_few_bases=bases.iloc[:10,:]\ndisplay(first_few_bases)\n\n\n\nfirst_few_bases['data']=first_few_bases.path.apply(lambda x:steps_to_df(x,x+'\\series_step_0.hdf5'))\ndisplay(first_few_bases.columns)\n\n\n#display(first_few_bases)\nfor index in first_few_bases.index:\n    create_multiplot(first_few_bases,index)\n"

In [23]:
"""display(first_few_bases.iloc[0,0])


#display(bases)
first_few_bases=bases.iloc[31:36,:]
display(first_few_bases)


first_few_bases['data']=first_few_bases.path.apply(lambda x:steps_to_df(x,x+'\\series_step_0.hdf5'))


display(first_few_bases)
for index in first_few_bases.index:
    create_multiplot(first_few_bases,index)"""





"display(first_few_bases.iloc[0,0])\n\n\n#display(bases)\nfirst_few_bases=bases.iloc[31:36,:]\ndisplay(first_few_bases)\n\n\nfirst_few_bases['data']=first_few_bases.path.apply(lambda x:steps_to_df(x,x+'\\series_step_0.hdf5'))\n\n\ndisplay(first_few_bases)\nfor index in first_few_bases.index:\n    create_multiplot(first_few_bases,index)"

In [24]:
display(bases)
first_few_bases=bases.iloc[:,:]
display(first_few_bases)

,path,data
MC_A_6xi,Y:\Istra Repositories\ch-gr_exports\MC_A_6xi,NaN
MC_A_6xiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6xiii,NaN
MC_A_6yi,Y:\Istra Repositories\ch-gr_exports\MC_A_6yi,NaN
MC_A_6yii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yii,NaN
MC_A_6yiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yiii,NaN
...,...,...
QL_GW_8xi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xi,NaN
QL_GW_8xii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xii,NaN
QL_GW_8xiii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xiii,NaN
QL_GW_8yi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8yi,NaN


,path,data
MC_A_6xi,Y:\Istra Repositories\ch-gr_exports\MC_A_6xi,NaN
MC_A_6xiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6xiii,NaN
MC_A_6yi,Y:\Istra Repositories\ch-gr_exports\MC_A_6yi,NaN
MC_A_6yii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yii,NaN
MC_A_6yiii,Y:\Istra Repositories\ch-gr_exports\MC_A_6yiii,NaN
...,...,...
QL_GW_8xi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xi,NaN
QL_GW_8xii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xii,NaN
QL_GW_8xiii,Y:\Istra Repositories\ch-gr_exports\QL_GW_8xiii,NaN
QL_GW_8yi,Y:\Istra Repositories\ch-gr_exports\QL_GW_8yi,NaN


In [ ]:
first_few_bases['data']=first_few_bases.path.apply(lambda x:steps_to_df(x,x+'\\series_step_0.hdf5'))
display(first_few_bases.columns)

In [ ]:
#display(first_few_bases)
for index in first_few_bases.index:
    create_multiplot(first_few_bases,index)

In [ ]:
print(first_few_bases.index)

In [ ]:
print([type(bases.loc[i,'data']) for i in bases.index])

In [ ]:
#bases.to_pickle('result',compression='zip')

In [ ]:
#topt=pd.read_pickle('result',compression ='zip')

In [ ]:
#display(topt.columns)